# PWP ocean model

### Basic overview

With the most basic set-up (i.e. one that uses prescribed surface fluxes and never calls the ice-model), the order of operations is as follows:

1. Set model parameters and configuration (`PWP_helper.set_params()`). 
2. Prepare forcing and profile data for model run (`PWP_helper.prep_data()`).
3. Iterate the PWP model (`PWP.pwpgo()`):
    + apply heat and E-P fluxes.
    + apply wind stress (momentum flux).
    + apply drag associated with internal wave dissipation.
    + apply bulk Richardson mixing.
    + apply gradient Richardson mixing. 
    + apply diapycnal diffusion.
4. Save results to output file (`PWP_helper.run_PWP()`).
5. Make simple plots to visualize the results (`PWP_helper.makeSomePlots()`). 


Examples of how to set-up and execute basic PWP runs are demonstrated in `demo1()` and `demo2()` in `PWP_demos.py`. 

### Input data

The PWP model requires *two input files*: one for the surface forcing and another for the initial profile - both files are expected to be netCDF. By default, the code simply applies the prescribed heat and surface to the T-S profile. With this configuration, the following data must be included in the surface forcing file:

+ **time**: time (fraction of day).
+ **sw**: net shortwave radiation (W/m<sup>2</sup>)
+ **lw**: net longwave radiation (W/m<sup>2</sup>)
+ **qlat**: latent heat flux (W/m<sup>2</sup>)
+ **qsens**: sensible heat flux (W/m<sup>2</sup>)
+ **tx**: eastward wind stress (N/m<sup>2</sup>)
+ **ty**: northward wind stress (N/m<sup>2</sup>)
+ **precip**: precipitation rate (m/s)

All are expected to be 1-D arrays of the same length. For the heat fluxes, **positive values should correspond to heat gained by the ocean**. Note that the MATLAB version of this code uses a opposite sign convention. 

The time data field should contain a 1-D array representing fraction of day. For example, for 6 hourly data, this should be a number series that increases in steps of 0.25, such as np.array([1.0, 1.25, 1.75, 2.0, 2.25...]).

The initial profile file should have the following data fields:
 
+ **z**: 1-D array of depth levels (m) 
+ **t**: 1-D array containing temperature profile (degrees celsius)
+ **s**: 1-D array containing salinity profile (PSU) 
+ **lat**: float representing latitude of profile

Examples of both input files are provided in the input directory.

Before starting the integration, the model interpolates the surface forcing and initial profile data to match the prescribed time step and depth increment, respectively. For the surface forcing, the model uses a simple linear interpolation. For the initial profile, the model uses `InterpolatedUnivariateSpline()` from `scipy.interpolate`. This is used to extrapolate the initial profile to the surface at 0m.


### Run configurations

The `set_params()` function in *PWP_helper.py* provide many options for configuring a model run. To display the defaults, run `PWP_helper.set_params(display_params=True)`. This prints all the parameters to the screen. Some important default settings are listed below:

+ **dt**: time-step increment in units of hours [3 hours]
+ **dz**: depth increment (meters). [1m]
+ **max_depth**: Max depth of vertical coordinate (meters). [100]
+ **mld_thresh**: Density criterion for MLD (kg/m3). [1e-4]
+ **dt_save**: time-step increment for saving to file (multiples of dt). [1]
+ **winds_ON**: toggle switch for wind forcing. [True]
+ **emp_ON**: toggle switch for freshwater forcing. [True]
+ **drag_ON**: toggle switch for current drag due to internal-inertial wave dispersion. [True]
+ **ice_ON**: toggle switch to allow sea-ice formation is surface layer temperature drops below freezing. [True]
+ **rkz**: vertical diffusivity coefficient (m**2/s). [0.]
+ **dens_option**: density option: 'dens0', 'pdens' or 'dens'. ['dens0']

For a full list of the default parameters, you can run `PWP_helper.set_params(display_params=True)`. This prints all the default configurations to the terminal. The default settings can be changed using the `param_mods` option in `run_PWP`. This is demonstrated in the example linked at the end.

For the density option, 'dens0' calls `dens0()` from the `seawater` module which computes density using in situ temperatures and salinity assuming surface pressure. In other words, this option ignores all pressure effects on density. Specifying 'pdens' calls `pden()` from the seawater module and computes potential density referenced to the surface. Specifying 'dens' calls `dens()` from the seawater module and computes density using the full density equation.

The default 'dens0' setting is the fastest option but it may not be diserable when working with weakly stratified water columns. In such scenarios, 'pdens' or 'dens' may be more appropriate. 'pdens' has a similar behavior to 'dens0' so it should be the first alternative. **Using full density should work in principle but the model has not undergone much testing with this option.**



### Initial stabilization

Intial T,S profiles can sometimes have instabilities, especially if they come from raw profile data. These instabilities are often caused by instrument noise and are usually inconsequential. However, the PWP mixing algorithm, specifically the `remove_si` function, is designed to mix away all instabilities. If that noise-generated instability is deep in the water column, `remove_si()` will irreparably deepen the mixed layer to that level on the very first time-step.

To circumvent this behavior, I implemented the `local_stir()` algorithm. This algorithm is based on the `stir()` algorithm used by `grad_mix()`. It is designed to eliminate small instabilities within the water column by mixing the unstable cell with an adjacent cell. The stabilization is done iteratively and starts out with a small mixing fraction (0.5). This mixing fraction is gradually increased with each iteration until all instabilities are removed. 

As a check, a comparison between the initial and stabilized plots are displayed, which the user has to manually approve before the model integration starts. This behavior can be disabled by setting `params['examine_stabilized_plot'] = False` in the `set_params()`.

### Runs with computed heat fluxes

The PWP code can also compute heat fluxes using bulk formula. These computations are done in `get_atm_ocean_HF()` in *PWP.py*. To exercise this option, set `use_Bulk_Formula = True` in `set_params()` or via the `param_mods` argument in `run_PWP` as demonstrated earlier. Additionally, include the following atmospheric state variables in the forcing file:

+ **dlw**: downwelling longwave radiation (W/m<sup>2</sup>)
+ **u10m**: zonal winds at 10m (m/s)
+ **v10m**: meridonial winds at 10m (m/s)
+ **shum2m**: specific humidity at 2m (kg/kg)
+ **atemp2m**: temperature at 2m (K)

Currently, the code is only able to compute *sensible heat*, *latent heat* and *upward longwave radiation*. This is done at beginning of every time step using the atmospheric state variables listed above plus the model derived SST. 

The turbulent heat fluxes are computed using the procedure outlined in [Large and Yeager 2009](https://link.springer.com/article/10.1007/s00382-008-0441-3) without any height adjustment to the temperature and humidity fields. The upward longwave heat flux is computed using Boltzman's law $ulw=\epsilon \sigma T^4$, where $\epsilon=1$ is the emissivity. The computed upward longwave flux is combined with the prescribed downwelling longwave flux to produce net longwave radiation.

When this option is set, the ice-atmosphere heat fluxes are also computed in a similar fashion. This is done in `get_atm_ice_HF()` in *PWP.py*.


### Examples

+ [Southern Ocean in the summer](https://github.com/earlew/pwp_python/blob/with_sea_ice_v2/Example_run_1.ipynb)